# Light GBM Model



In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
import warnings
warnings.filterwarnings('ignore')

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier



FEATURE_SELECTION = False
tgt_encode = False

random.seed(random.randint(0, 100000))

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)


In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")
print(train_df.shape)
print(test_df.shape)

(66125, 304)
(28340, 303)


In [3]:
y = train_df["y"].values
train_df = train_df.drop("y", axis=1)

In [4]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

train_df["A1-level"] = train_df["A1-level"].astype(float)
test_df["A1-level"] = test_df["A1-level"].astype(float)
train_df["B1-level"] = train_df["B1-level"].astype(float)
test_df["B1-level"] = test_df["B1-level"].astype(float)

In [5]:
# add info about numeric column
# 武器データの中のnum_colsを持つ特徴について、チーム内の統計量を計算、levelは結構効いてる

# num_cols = ["level", "range", "rapid", "atack", "ink-sub", "fav-main", "good-special"]

num_cols = [
    "level", "range-main", "range-bullet-main", "distant-range_sub", 
    "rapid", "atack", "ink-sub", "fav-main", "good-special", "DPS", "kill_time_ika-main",
    "front_gap_human-main", "front_gap_ika-main", "rensya_frame-main", "saidai_damege-main", "damage_min-sub", 
    "damage_max-sub", "install_num-sub", "good-sub", "damage_max-special", 
    "duration-special", "good-special", "direct_rad-special", "distant_rad-special"
]

cols = [col for col in train_df.columns if "A1" in col or "A2" in col or "A3" in col or "A4" in col or 
        "B1" in col or "B2" in col or "B3" in col or "B4" in col]
drop_cols = []
for col1 in cols:
    f = True
    for col2 in num_cols:
        if col2 in col1:
            f = False
    if f and train_df[col1].dtype in [int, float]:
        drop_cols.append(col1)
        
train_df = train_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)
        
    

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)

# 欠損値埋め先にやろう
欠損値がある列

rank, weapon, level ⇨　回線切れ or ナワバリ

weaponの欠損に並んでそれに関係ある列、A3, A4, B3, B4 ⇨ 回線切れ

level, weaponが消えていたら回線切れ

In [6]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [7]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

0
0


# 特徴量エンジニアリング


In [8]:
# count reskin, by mode
print("reskin")
train_df, test_df = prepro.count_reskin(train_df, test_df)
train_df, test_df = prepro.count_reskin_by_mode(train_df, test_df)

# count mainweapon, by mode
print("mainweapon")
train_df, test_df = prepro.count_mainweapon(train_df, test_df)
train_df, test_df = prepro.count_mainweapon_by_mode(train_df, test_df)

# count subweapon, by mode
print("subweapon")
train_df, test_df = prepro.count_subweapon(train_df, test_df)
train_df, test_df = prepro.count_subweapon_by_mode(train_df, test_df)

# count special, by mode
print("special")
train_df, test_df = prepro.count_special(train_df, test_df)
train_df, test_df = prepro.count_special_by_mode(train_df, test_df)


#identify A1
train_df, test_df = prepro.identify_A1(train_df, test_df)


# 水増し, A1も統計量に含めた特徴を作る場合は水ましより先にやる
print("mizumashi")
train_df, y = prepro.mizumashi(train_df, y)

# is_nawabari
train_df, test_df = prepro.is_nawabari(train_df, test_df)

# match rank、単体で意味なし
train_df, test_df = prepro.match_rank(train_df, test_df)

# rankを二列に分ける
train_df, test_df = prepro.ranker(train_df, test_df)




reskin
mainweapon
subweapon
special
mizumashi


In [9]:
# add team info、メインはなくてもいい

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon", "rank-mark"])
#train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "rank-mark"])

(160590, 554)
special-A
special-B
subweapon-A
subweapon-B
category1-A
category1-B
category2-A
category2-B
mainweapon-A
mainweapon-B
rank-mark-A
rank-mark-B
(160590, 758)
complete


In [10]:
# categorize team , 良い

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
#categorize_col = ["category1", "category2", "subweapon", "special"]
for col in categorize_col:
    print(col)
    train_df, test_df = prepro.categorize_team(train_df, test_df, col)

category1
category2
subweapon
special
mainweapon


In [11]:
# product categorical feature
train_df, test_df = prepro.prod(train_df, test_df, "mode", "stage")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "match_rank")

train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "match_rank")

In [12]:
# make input
drop_cols = [
    "id", "lobby", "lobby-mode",  "period", "game-ver", "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", 
    "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon", 
    "A-a-rank-mark-onehot", "A-o-rank-mark-onehot", 
    "B-a-rank-mark-onehot", "B-o-rank-mark-onehot",
    #"reskin-A1", "reskin-A2", "reskin-A3", "reskin-A4", 
    #"reskin-B1", "reskin-B2", "reskin-B3", "reskin-B4",
    #"category1-A1", 
    #"category2-A2", 
    #"mainweapon-A1",
    #"subweapon-A1",
    #"special-A1",
]

train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)



#X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=False, scaler=False, verbose=False)

In [13]:
cat_col = [col for col in train_df.columns if train_df[col].dtype == "object"]
all_df = pd.concat([train_df, test_df])
for col in cat_col:
    print(col)
    li = all_df[col].unique()
    di = {}

    for i in range(0, len(li)):
        di[li[i]] = i

    train_df[col] = train_df[col].map(lambda x : di[x] % 10)

mode
stage
category1-A1
category2-A1
subweapon-A1
special-A1
mainweapon-A1
reskin-A1
category1-A2
category2-A2
subweapon-A2
special-A2
mainweapon-A2
reskin-A2
category1-A3
category2-A3
subweapon-A3
special-A3
mainweapon-A3
reskin-A3
category1-A4
category2-A4
subweapon-A4
special-A4
mainweapon-A4
reskin-A4
category1-B1
category2-B1
subweapon-B1
special-B1
mainweapon-B1
reskin-B1
category1-B2
category2-B2
subweapon-B2
special-B2
mainweapon-B2
reskin-B2
category1-B3
category2-B3
subweapon-B3
special-B3
mainweapon-B3
reskin-B3
category1-B4
category2-B4
subweapon-B4
special-B4
mainweapon-B4
reskin-B4
match_rank
rank-mark-A1
rank-mark-A2
rank-mark-A3
rank-mark-A4
rank-mark-B1
rank-mark-B2
rank-mark-B3
rank-mark-B4
team-category1-A
team-category1-B
team-category2-A
team-category2-B
team-subweapon-A
team-subweapon-B
team-special-A
team-special-B
team-mainweapon-A
team-mainweapon-B
mode x stage
mode x team-category1-A
mode x team-category1-B
mode x team-category2-A
mode x team-category2-B
mode 

In [14]:
train_df

,mode,stage,A1-level,A2-level,A3-level,A4-level,B1-level,B2-level,B3-level,B4-level,category1-A1,category2-A1,subweapon-A1,special-A1,mainweapon-A1,reskin-A1,range-main-A1,atack-main-A1,rapid-main-A1,fav-main-A1,DPS-main-A1,kill_time_ika-main-A1,front_gap_human-main-A1,front_gap_ika-main-A1,rensya_frame-main-A1,saidai_damege-main-A1,range-bullet-main-A1,damage_min-sub-A1,damage_max-sub-A1,ink-sub-A1,install_num-sub-A1,good-sub-A1,distant-range_sub-A1,damage_max-special-A1,duration-special-A1,good-special-A1,direct_rad-special-A1,distant_rad-special-A1,category1-A2,category2-A2,subweapon-A2,special-A2,mainweapon-A2,reskin-A2,range-main-A2,atack-main-A2,rapid-main-A2,fav-main-A2,DPS-main-A2,kill_time_ika-main-A2,front_gap_human-main-A2,front_gap_ika-main-A2,rensya_frame-main-A2,saidai_damege-main-A2,range-bullet-main-A2,damage_min-sub-A2,damage_max-sub-A2,ink-sub-A2,install_num-sub-A2,good-sub-A2,distant-range_sub-A2,damage_max-special-A2,duration-special-A2,good-special-A2,direct_rad-special-A2,distant_rad-special-A2,category1-A3,category2-A3,subweapon-A3,special-A3,mainweapon-A3,reskin-A3,range-main-A3,atack-main-A3,rapid-main-A3,fav-main-A3,DPS-main-A3,kill_time_ika-main-A3,front_gap_human-main-A3,front_gap_ika-main-A3,rensya_frame-main-A3,saidai_damege-main-A3,range-bullet-main-A3,damage_min-sub-A3,damage_max-sub-A3,ink-sub-A3,install_num-sub-A3,good-sub-A3,distant-range_sub-A3,damage_max-special-A3,duration-special-A3,good-special-A3,direct_rad-special-A3,distant_rad-special-A3,category1-A4,category2-A4,subweapon-A4,special-A4,mainweapon-A4,reskin-A4,range-main-A4,atack-main-A4,rapid-main-A4,fav-main-A4,DPS-main-A4,kill_time_ika-main-A4,front_gap_human-main-A4,front_gap_ika-main-A4,rensya_frame-main-A4,saidai_damege-main-A4,range-bullet-main-A4,damage_min-sub-A4,damage_max-sub-A4,ink-sub-A4,install_num-sub-A4,good-sub-A4,distant-range_sub-A4,damage_max-special-A4,duration-special-A4,good-special-A4,direct_rad-special-A4,distant_rad-special-A4,category1-B1,category2-B1,subweapon-B1,special-B1,mainweapon-B1,reskin-B1,range-main-B1,atack-main-B1,rapid-main-B1,fav-main-B1,DPS-main-B1,kill_time_ika-main-B1,front_gap_human-main-B1,front_gap_ika-main-B1,rensya_frame-main-B1,saidai_damege-main-B1,range-bullet-main-B1,damage_min-sub-B1,damage_max-sub-B1,ink-sub-B1,install_num-sub-B1,good-sub-B1,distant-range_sub-B1,damage_max-special-B1,duration-special-B1,good-special-B1,direct_rad-special-B1,distant_rad-special-B1,category1-B2,category2-B2,subweapon-B2,special-B2,mainweapon-B2,reskin-B2,range-main-B2,atack-main-B2,rapid-main-B2,fav-main-B2,DPS-main-B2,kill_time_ika-main-B2,front_gap_human-main-B2,front_gap_ika-main-B2,rensya_frame-main-B2,saidai_damege-main-B2,range-bullet-main-B2,damage_min-sub-B2,damage_max-sub-B2,ink-sub-B2,install_num-sub-B2,good-sub-B2,distant-range_sub-B2,damage_max-special-B2,duration-special-B2,good-special-B2,direct_rad-special-B2,distant_rad-special-B2,category1-B3,category2-B3,subweapon-B3,special-B3,mainweapon-B3,reskin-B3,range-main-B3,atack-main-B3,rapid-main-B3,fav-main-B3,DPS-main-B3,kill_time_ika-main-B3,front_gap_human-main-B3,front_gap_ika-main-B3,rensya_frame-main-B3,saidai_damege-main-B3,range-bullet-main-B3,damage_min-sub-B3,damage_max-sub-B3,ink-sub-B3,install_num-sub-B3,good-sub-B3,distant-range_sub-B3,damage_max-special-B3,duration-special-B3,good-special-B3,direct_rad-special-B3,distant_rad-special-B3,category1-B4,category2-B4,subweapon-B4,special-B4,mainweapon-B4,reskin-B4,range-main-B4,atack-main-B4,rapid-main-B4,fav-main-B4,DPS-main-B4,kill_time_ika-main-B4,front_gap_human-main-B4,front_gap_ika-main-B4,rensya_frame-main-B4,saidai_damege-main-B4,range-bullet-main-B4,damage_min-sub-B4,damage_max-sub-B4,ink-sub-B4,install_num-sub-B4,good-sub-B4,distant-range_sub-B4,damage_max-special-B4,duration-special-B4,good-special-B4,direct_rad-special-B4,distant_rad-special-B4,disconnection-A,disconnection-B,A-level-max,A-level-min,A-level-mean,A-level-median,A-level-std,A-range-main-max,A-range-mai

In [ ]:
all_df = pd.concat([train_df, test_df])
for col in cat_col:
    print(col)
    all_df = pd.concat([all_df, pd.get_dummies(train_df["mode"], prefix="mode", drop_first=True)])
    
all_df.drop(columns=cat_col, inplace=True)
train_df = all_df[:train_df.shape[0]]
test_df = all_df[train_df.shape[0]:]

mode
stage
category1-A1
category2-A1
subweapon-A1
special-A1
mainweapon-A1
reskin-A1


In [ ]:
float_col = [col for col in train_df.columns if train_df[col].dtype == float]
from sklearn.preprocessing import StandardScaler
for col in float_col:
    scaler = StandardScaler()
    print("scale")
    scaler.fit(all_df[col].values.reshape(-1, 1))
    train_df[col] = scaler.transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = scaler.transform(test_df[col].values.reshape(-1, 1))

In [ ]:
X = train_df
test_X = test_df
del all_df, train_df, test_df
print(X.shape)
print(test_X.shape)

# モデル学習

In [ ]:
# 全データを5つに分割
random.seed(random.randint(0, 10000))
SIZE = X.shape[0]
K = 5


folds = prepro.make_stratified_kfolds(X, y, K, shuffle=True)
#folds = prepro.make_stratified_kfolds(X, X["mode"] + y.astype(str), K, shuffle=True)


print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    
    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")
    
if FEATURE_SELECTION:
    X = X[use_cols]
    test_X = test_X[use_cols]

In [ ]:

THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []
train_Xs = []
valid_Xs = []


all_indices = sum(folds, [])
for i in range(K):
    print("======================== fold {} ========================".format(i+1))
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    train_X = X.iloc[train_indices]
    train_y = y[train_indices]
    valid_X = X.iloc[valid_indices]
    valid_y = y[valid_indices]
    
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(train_X, valid_X)
    
    pred = neigh.predict_proba(valid_X)[:,1]
    train_pred.append(pred)
    temp += np.sum(pred)
    pred = np.where(pred < THRESHOLD, 0, 1)
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

In [ ]:
preds = []
if tgt_encode:
    for col in cat_tgtenc_cols:
        print(col)
        _, test_X_ = prepro.target_encoder(X, test_X, col, y)
        if col in test_X.columns:
            test_X = test_X.drop(col, axis=1)
    test_X = prepro.add_numeric_info(test_X, ["tgtenc-mode x reskin"])


        
for i in range(K):
    model = models[i]
    pred = model.predict(test_X)
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":pred})
temp.hist(bins=80)

In [ ]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_{}.csv'.format(K))

# モデル解釈

In [ ]:
importance = pd.DataFrame(models[0].feature_importance(), index=train_Xs[0].columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance.iloc[:100])

In [ ]:
indices = [index for index in importance.index if "tgtenc" in index]
importance.loc[indices]

In [ ]:
importance["importance"].sum()

In [ ]:
train_df["pred"] = 0
train_df["y"] = y
for i in range(K):
    train_df["pred"].iloc[folds[i]] = train_pred[i]
for mode in train_df["mode"].unique():
    rate = train_df[train_df["mode"] == mode][train_df["pred"] == train_df["y"]].shape[0] / train_df[train_df["mode"] == mode].shape[0]
    print("{} : {}".format(mode, rate))
print(train_df[train_df["pred"] == train_df["y"]].shape[0] / train_df.shape[0])

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
result_df.to_csv("../../data/result.csv", index=False)

In [ ]:
print(train_df[train_df["mode"] == "nawabari"].shape[0]/train_df.shape[0])
print(train_df[train_df["mode"] == "hoko"].shape[0]/train_df.shape[0])
print(train_df[train_df["mode"] == "asari"].shape[0]/train_df.shape[0])
print(train_df[train_df["mode"] == "area"].shape[0]/train_df.shape[0])
print(train_df[train_df["mode"] == "yagura"].shape[0]/train_df.shape[0])

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df.loc[:100]

In [ ]:
num_cols_ = [
    "range-main", "range-bullet-main", "range-draw-main", "direct_range-sub", "distant-range_sub", 
    "rapid", "atack", "ink-sub", "fav-main", "good-special", "DPS", "kill_time_human-main", "kill_time_ika-main",
    "front_gap_human-main", "front_gap_ika-main", "rensya_frame-main", "saidai_damege-main", "damage_min-sub", 
    "damage_max-sub", "install_num-sub", "good-sub", "direct_range-sub", "damage_max-special", 
    "damage_min-special", "duration-special", "good-special", "direct_rad-special", "close_rad-special", "distant_rad-special"
]


In [ ]:
feature_cols = []
for name in num_cols_:
    for col in X.columns:
        if name in col:
            feature_cols.append(col)
                
importance.loc[feature_cols]#.sort_values('importance', ascending=False)

In [ ]:
sss = []

"""
A-direct_range-sub-mean 30
A-distant_range_sub-max
B-rapid-std	52
ink-sub
A-kill_time_human-main-median	36
A-kill_time_ika-main-median	58
A-front_gap_human-main-std	25
A-front_gap_ika-main-std	48
A-saidai_damege-main-mean	40
damage_max-sub 37
install_num-sub 10
direct_range-sub 30
damage_max-special 47
duration-special 46
close_rad-special 30
distant_rad-special 21
"""




for col in X.columns:
    if "distant_rad-special" in col:
        sss.append(col)
                
importance.loc[sss].sort_values('importance', ascending=False)

In [ ]:
li = [2, 3, 4, 5, 6, 7]

In [ ]:
di = dict()
for i in range(0, len(li)):
    di[li[i]] = i

In [ ]:
di